In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from zipfile import ZipFile
from pathlib import Path

In [3]:
path = Path().parent.absolute()

In [4]:
# Extract .zip file download links from webrobots.io and append to an interable.
url = 'https://webrobots.io/kickstarter-datasets/'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
zip_urls = []
for link in soup.find_all('a'):
    try:
        if 'Z.zip' in link.get('href'):
            zip_urls.append(link.get('href'))
    except:
        continue

In [ ]:
# Define necessary directoriess, loop through .zip file list created in step above, dowload .CSVs and append to df. 
path = Path().parent.absolute()
temp_zip = str(path)+'/temp.zip'

fdf = pd.DataFrame()

for zipurl in zip_urls:
    zipresp = urlopen(zipurl)
    tempzip = open(temp_zip, "wb")
    tempzip.write(zipresp.read())
    tempzip.close()
    zf = zipfile.ZipFile(temp_zip)
    
    text_files = zf.infolist()
    for text_file in text_files:
        try:
            df = pd.read_csv(zf.open(text_file.filename))
            df['pull_date'] = zip_file
            df = df[df['source_url'].str.contains('categories/games')]
            fdf = fdf.append(df)
        except:
            continue
    
    zf.close()

os.remove("temp.zip")

In [ ]:
adf = fdf.drop(
         ['currency_symbol', 
          'currency_trailing_code',
          'current_currency',
          'disable_communication',
          'is_backing',
          'permissions',
          'urls',
          'unread_messages_count',
          'unseen_activity_count',
          'country_displayable_name',
          'projects',
          'total_hits',
          'seed',
          'search_url',
          'has_more',
          'last_update_published_at'
         ]
         , axis=1)

adf['created_at'] = pd.to_datetime(adf['created_at'].fillna(0).astype('int'),unit='s')
adf['deadline'] = pd.to_datetime(adf['deadline'].fillna(0).astype('int'),unit='s')
adf['launched_at'] = pd.to_datetime(adf['launched_at'].fillna(0).astype('int'),unit='s')
adf['state_changed_at'] = pd.to_datetime(adf['state_changed_at'].fillna(0).astype('int'),unit='s')

adf['pull_date'] = adf['pull_date'].str.replace('Kickstarter_', '').str.replace('Z.zip', '')
adf['pull_date'] = pd.to_datetime(adf['pull_date'], format='%Y-%m-%dT%H_%M_%S_%f')

In [ ]:
idx = adf.groupby(['slug'])['pull_date'].transform(max) == adf['pull_date']
adf = adf[idx]
adf = adf.drop_duplicates(subset=['slug'])

In [ ]:
adf.to_csv(str(path)+'/KICK_Data.csv')